In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
pd.set_option('display.max_columns', 500)

---